In [1]:
# Import libraries
import requests
from bs4 import BeautifulSoup
import urllib.request as ur
import time
import lxml.html as lh
header = {'User-Agent': 'Mozilla/5.0'}

In [2]:
#read and open URL to scrape
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url) #Connect to the URL
response #200 means it went through

<Response [200]>

In [3]:
#Parse HTML and save to BeautifulSoup object
soup = BeautifulSoup(response.text, "html.parser")
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className=document.documentElement.className.replace(/(^|\s)client-nojs(\s|$)/,"$1client-js$2");RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":906439794,"wgRevisionId":906439794,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1 errors: deprecated parameters","Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":!1,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","Feb

In [4]:
tables = soup.findAll('table',{'class':'wikitable sortable'})
tables

[<table class="wikitable sortable">
 <tbody><tr>
 <th>Postcode</th>
 <th>Borough</th>
 <th>Neighbourhood
 </th></tr>
 <tr>
 <td>M1A</td>
 <td>Not assigned</td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M2A</td>
 <td>Not assigned</td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M3A</td>
 <td><a href="/wiki/North_York" title="North York">North York</a></td>
 <td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
 </td></tr>
 <tr>
 <td>M4A</td>
 <td><a href="/wiki/North_York" title="North York">North York</a></td>
 <td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
 </td></tr>
 <tr>
 <td>M5A</td>
 <td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
 <td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
 </td></tr>
 <tr>
 <td>M5A</td>
 <td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
 <td><a href="/wiki/Regent_Park" title="Regent Park">Regent Pa

In [5]:
#How many tables are in the HTML text
for table in tables: 
    print("#########")
    print(table.text[:100])

#########


Postcode
Borough
Neighbourhood


M1A
Not assigned
Not assigned


M2A
Not assigned
Not assigned


M


In [6]:
for tn in range(len(tables)):
    table = tables[tn]

In [7]:
#Parse tr/td text and compile into rows
rows = table.findAll("tr")
row_lengths = [len(r.findAll(['th', 'td'])) for r in rows]
ncols = max(row_lengths)
nrows = len(rows)
data = []
for i in range(nrows):
    rowD = []
    for j in range(ncols):
        rowD.append('')
    data.append(rowD)

In [8]:
for i in range(len(rows)):
    row = rows[i]
    rowD = []
    cells = row.findAll(["td", "th"])
    for j in range(len(cells)):
        cell = cells[j]
    #lots of cells span cols and rows so lets deal with that
        cspan = int(cell.get('colspan', 1))
        rspan = int(cell.get('rowspan', 1))
        l = 0
        for k in range(rspan):
            # Shifts to the first empty cell of this row
            while data[i + k][j + l]:
                l += 1
            for m in range(cspan):
                data[i + k][j + l + m] += cell.text
                    
    data.append(rowD)

In [9]:
len(data)

578

In [10]:
import os
import codecs

In [11]:
 # write data out to tab seperated format
page = os.path.split(url)[1]
fname = 'output_{}_t{}.tsv'.format(page, tn)
f = codecs.open(fname, 'w')
for i in range(nrows):
    rowStr = '\t'.join(data[i])
    rowStr = rowStr.replace('\n', '')
    print(rowStr)
    f.write(rowStr + '\n')

Postcode	Borough	Neighbourhood
M1A	Not assigned	Not assigned
M2A	Not assigned	Not assigned
M3A	North York	Parkwoods
M4A	North York	Victoria Village
M5A	Downtown Toronto	Harbourfront
M5A	Downtown Toronto	Regent Park
M6A	North York	Lawrence Heights
M6A	North York	Lawrence Manor
M7A	Queen's Park	Not assigned
M8A	Not assigned	Not assigned
M9A	Etobicoke	Islington Avenue
M1B	Scarborough	Rouge
M1B	Scarborough	Malvern
M2B	Not assigned	Not assigned
M3B	North York	Don Mills North
M4B	East York	Woodbine Gardens
M4B	East York	Parkview Hill
M5B	Downtown Toronto	Ryerson
M5B	Downtown Toronto	Garden District
M6B	North York	Glencairn
M7B	Not assigned	Not assigned
M8B	Not assigned	Not assigned
M9B	Etobicoke	Cloverdale
M9B	Etobicoke	Islington
M9B	Etobicoke	Martin Grove
M9B	Etobicoke	Princess Gardens
M9B	Etobicoke	West Deane Park
M1C	Scarborough	Highland Creek
M1C	Scarborough	Rouge Hill
M1C	Scarborough	Port Union
M2C	Not assigned	Not assigned
M3C	North York	Flemingdon Park
M3C	North York	Don Mills South
M

In [12]:
import pandas as pd

df = pd.read_csv(fname, sep='\t', header=0)

df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [13]:
import numpy as np

In [14]:
# Drop rows with not assigned Borough
df2 = df[~df['Borough'].str.contains('Not assigned')]
df2

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [15]:
# Replace unassigned Neighbourhoods with Borough names
for i in range(df2.shape[0]):
    if df2.iloc[i,:]['Neighbourhood']=='Not assigned':
        df2.iloc[i,:]['Neighbourhood'] = df2.iloc[i,:]['Borough']
df2.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [16]:
df2.shape

(211, 3)

In [17]:
df3_grouped = df2.groupby(['Postcode','Borough'], as_index=False).agg(lambda x: ','.join(x))

df3_grouped.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [18]:
df3_grouped.shape

(103, 3)

In [19]:
path = 'C:\\Users\\LeaMPalmer\\Documents\\Notes\\Python and Data Science\\CSV Files\\Geospatial_Coordinates.csv'
df_coord = pd.read_csv(path, header='infer')
df_coord.rename(columns={'Postal Code':'Postcode'},inplace='True')
df_coord.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
df4_lat_lng = df3_grouped.merge(df_coord, on='Postcode', how='inner')
df4_lat_lng.head(30)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [21]:
print('The dataframe has {} boroughs and {} neighbourhoods.'.format(
        len(df4_lat_lng['Borough'].unique()),
        df4_lat_lng.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighbourhoods.


In [22]:
df4_lat_lng.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [23]:
# !conda install -c conda-forge folium=0.5.0 --yes

In [24]:
# pip install geopy

In [25]:
from geopy.geocoders import Nominatim
import folium

In [26]:
# Grab the coordinates of Toronto
address="Toronto, Canada"

geolocator = Nominatim(user_agent="exploring_toronto_map")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinates of Toronto are 43.653963, -79.387207.


In [27]:
toronto_map=folium.Map(location=[latitude,longitude],zoom_start=10)

for lat, lng, label in zip(df4_lat_lng['Latitude'], df4_lat_lng['Longitude'], df4_lat_lng['Borough']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='coral',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.8,
        parse_html=False).add_to(toronto_map)
    
toronto_map

In [28]:
scarborough_data = df4_lat_lng[df4_lat_lng['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [29]:
address = 'Scarborough, Canada'

geolocator = Nominatim(user_agent="exploring_scarborough")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Scarborough are {}, {}.'.format(latitude, longitude))

The geographical coordinates of Scarborough are 43.773077, -79.257774.


In [30]:
scarborough_map=folium.Map(location=[latitude,longitude],zoom_start=11)

for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(scarborough_map)
    
scarborough_map

In [31]:
CLIENT_ID = 'IMHULEGENXQIE2V42EYXQ5GOO0IFTFMKBDZQDMFI0I3DLFFR'
CLIENT_SECRET = 'ODZRXGFQCB22JOCVJLYUVSAANRGSRCF3G40PHFJ5RSXVZTSD'
VERSION = '20180605'


print('Your Credentials')
print('Client_ID: '+ CLIENT_ID)
print('Client Secret: ' + CLIENT_SECRET)

Your Credentials
Client_ID: IMHULEGENXQIE2V42EYXQ5GOO0IFTFMKBDZQDMFI0I3DLFFR
Client Secret: ODZRXGFQCB22JOCVJLYUVSAANRGSRCF3G40PHFJ5RSXVZTSD


In [32]:
scarborough_data.loc[0, 'Neighbourhood']

'Rouge,Malvern'

In [33]:
neighbourhood_latitude = scarborough_data.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = scarborough_data.loc[0, 'Longitude'] # neighborhood longitude value
neighbourhood_name = scarborough_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Rouge,Malvern are 43.806686299999996, -79.19435340000001.


In [34]:
search_query= 'Rouge,Malvern'
radius=50000
LIMIT = 100

url='https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighbourhood_latitude, neighbourhood_longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=IMHULEGENXQIE2V42EYXQ5GOO0IFTFMKBDZQDMFI0I3DLFFR&client_secret=ODZRXGFQCB22JOCVJLYUVSAANRGSRCF3G40PHFJ5RSXVZTSD&ll=43.806686299999996,-79.19435340000001&v=20180605&query=Rouge,Malvern&radius=50000&limit=100'

In [35]:
results = requests.get(url).json()
venues=results['response']['venues']
venues

[{'id': '4d5475dbcc65a1431140425e',
  'name': 'Malvern Rouge Valley Youth Services',
  'location': {'address': '1270 Morningside Ave, Unit 5',
   'lat': 43.797791,
   'lng': -79.198267,
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.797791,
     'lng': -79.198267}],
   'distance': 1038,
   'cc': 'CA',
   'country': 'Canada',
   'formattedAddress': ['1270 Morningside Ave, Unit 5', 'Canada']},
  'categories': [{'id': '52e81612bcbc57f1066b7a34',
    'name': 'Community Center',
    'pluralName': 'Community Centers',
    'shortName': 'Community Center',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/default_',
     'suffix': '.png'},
    'primary': True}],
  'referralId': 'v-1568751893',
  'hasPerk': False},
 {'id': '4bb265de715eef3b5e2c85bb',
  'name': 'Toronto Public Library - Malvern Branch',
  'location': {'address': '30 Sewells Rd.',
   'crossStreet': 'at Brenyon Way',
   'lat': 43.80806841399391,
   'lng': -79.21615264459346,
   'labeledLatLngs': [{

In [36]:
# Function that extracts the category of the venue(s) #
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [37]:
import json 
from pandas.io.json import json_normalize

In [38]:
nearby_venues = json_normalize(venues)
nearby_venues.columns

Index(['categories', 'hasPerk', 'id', 'location.address', 'location.cc',
       'location.city', 'location.country', 'location.crossStreet',
       'location.distance', 'location.formattedAddress',
       'location.labeledLatLngs', 'location.lat', 'location.lng',
       'location.neighborhood', 'location.postalCode', 'location.state',
       'name', 'referralId'],
      dtype='object')

In [39]:
# filter columns
filtered_columns = ['name', 'categories', 'location.lat', 'location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Malvern Rouge Valley Youth Services,Community Center,43.797791,-79.198267
1,Toronto Public Library - Malvern Branch,Library,43.808068,-79.216153
2,Malvern Woods Park,Park,43.802091,-79.203851
3,Rouge Valley Centenary Hospital,Hospital,43.780614,-79.205151
4,Rouge Hill GO Station,Train Station,43.780229,-79.130855


In [40]:
#How many venues were returned by Foursquare
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

50 venues were returned by Foursquare.


In [41]:

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [42]:
toronto_venues = getNearbyVenues(names=df4_lat_lng['Neighbourhood'],
                                   latitudes=df4_lat_lng['Latitude'],
                                   longitudes=df4_lat_lng['Longitude']
                                  )

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

In [43]:
print (toronto_venues.shape)
toronto_venues.head()
toronto_venues.index

(2254, 7)


RangeIndex(start=0, stop=2254, step=1)

In [44]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",3,3,3,3,3,3
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",9,9,9,9,9,9
"Alderwood,Long Branch",10,10,10,10,10,10
"Bathurst Manor,Downsview North,Wilson Heights",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",22,22,22,22,22,22
Berczy Park,55,55,55,55,55,55


In [45]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 277 uniques categories.


In [46]:
toronto_venues[:5]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa


In [47]:
feature_df = pd.DataFrame()

for nh in set(toronto_venues['Neighborhood']):
    nh_subset = toronto_venues[toronto_venues['Neighborhood']==nh]
    
    test = nh_subset.groupby('Venue Category').count().reset_index()[['Venue Category','Neighborhood']]
    test.columns = ['Venue','Num']
    test = test.transpose()
    test.columns = test.values[0]
    test = test[1:].reset_index(drop=True)
    
    test['neighborhood'] = nh
    test['Lat'] = nh_subset['Neighborhood Latitude'].values[0]
    test['Long'] = nh_subset['Neighborhood Longitude'].values[0]
    
    feature_df = feature_df.append(test)

C:\Users\LeaMPalmer\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [48]:
num_yoga_df = feature_df[['neighborhood','Yoga Studio']]
num_yoga_df.shape

(99, 2)

In [49]:
feature_df.shape

(99, 280)

In [50]:
# Drop cols where most are NaN
feature_df = feature_df.dropna(thresh=round(feature_df.shape[0]*0.25), axis=1).reset_index(drop=True)
feature_df.shape

(99, 11)

In [51]:
# now fill the remainig
feature_df

,Bakery,Café,Coffee Shop,Italian Restaurant,Lat,Long,Park,Pizza Place,Restaurant,Sandwich Place,neighborhood
0,NaN,NaN,NaN,NaN,43.706748,-79.594054,NaN,NaN,NaN,NaN,Northwest
1,1,2,3,1,43.647927,-79.419750,1,2,2,NaN,"Little Portugal,Trinity"
2,NaN,NaN,NaN,NaN,43.685347,-79.338106,2,NaN,NaN,NaN,East Toronto
3,NaN,2,3,NaN,43.770120,-79.408493,NaN,2,2,2,Willowdale South
4,1,4,12,4,43.657952,-79.387383,1,NaN,NaN,3,Central Bay Street
5,2,NaN,5,NaN,43.778517,-79.346556,NaN,NaN,1,1,"Fairview,Henry Farm,Oriole"
6,NaN,NaN,NaN,NaN,43.696948,-79.411307,NaN,NaN,NaN,NaN,"Forest Hill North,Forest Hill West"
7,1,NaN,NaN,NaN,43.713756,-79.490074,1,NaN,NaN,NaN,"Downsview,North Park,Upwood Park"
8,1,NaN,NaN,NaN,43.750072,-79.295849,NaN,NaN,NaN,NaN,"Maryvale,Wexford"
9,2,4,13,2,43.640816,-79.381752,2,2,2,1,"Harbourfront East,Toronto Islands,Union Station"


In [52]:
feature_df = feature_df.fillna(0)
feature_df[:5]

,Bakery,Café,Coffee Shop,Italian Restaurant,Lat,Long,Park,Pizza Place,Restaurant,Sandwich Place,neighborhood
0,0,0,0,0,43.706748,-79.594054,0,0,0,0,Northwest
1,1,2,3,1,43.647927,-79.419750,1,2,2,0,"Little Portugal,Trinity"
2,0,0,0,0,43.685347,-79.338106,2,0,0,0,East Toronto
3,0,2,3,0,43.770120,-79.408493,0,2,2,2,Willowdale South
4,1,4,12,4,43.657952,-79.387383,1,0,0,3,Central Bay Street


In [53]:
from sklearn.cluster import KMeans

In [54]:
#set number of kmeans clusters
kclusters = 5

# Before you cluster~!:
# Be sure to save the number of yoga studios as a new data frame and then DROP/REMOVE it from your existing dataframe
# This way you're not biasing your clustering algorith.
# aka: when deciding where to put yoga studios, you don't want to consider how many there currently are.

# toronto_clustered_groups = toronto_grouped.drop('Neighborhood', 1)

#run kmeans clusters
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(feature_df[feature_df.columns[:-1]])

# #check cluster labels
# kmeans.labels_[0:10]

In [55]:
kmeans

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=0, tol=0.0001, verbose=0)

In [56]:
kmeans.labels_

array([2, 0, 2, 0, 1, 3, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0,
       4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 3, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 0, 2, 2, 4, 2, 2, 0, 4, 2, 2, 2, 0, 4, 2, 0, 2, 0, 2, 2,
       2, 2, 2, 2, 2, 4, 3, 2, 2, 3, 2, 2, 3, 4, 2, 2, 2, 0, 2, 0, 2, 0,
       2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 0])

In [57]:
feature_df['Cluster'] = kmeans.labels_

In [58]:
feature_df = feature_df.merge(num_yoga_df, how='inner', on='neighborhood')

In [59]:
feature_df = feature_df.fillna(0)
feature_df

,Bakery,Café,Coffee Shop,Italian Restaurant,Lat,Long,Park,Pizza Place,Restaurant,Sandwich Place,neighborhood,Cluster,Yoga Studio
0,0,0,0,0,43.706748,-79.594054,0,0,0,0,Northwest,2,0
1,1,2,3,1,43.647927,-79.419750,1,2,2,0,"Little Portugal,Trinity",0,1
2,0,0,0,0,43.685347,-79.338106,2,0,0,0,East Toronto,2,0
3,0,2,3,0,43.770120,-79.408493,0,2,2,2,Willowdale South,0,0
4,1,4,12,4,43.657952,-79.387383,1,0,0,3,Central Bay Street,1,1
5,2,0,5,0,43.778517,-79.346556,0,0,1,1,"Fairview,Henry Farm,Oriole",3,0
6,0,0,0,0,43.696948,-79.411307,0,0,0,0,"Forest Hill North,Forest Hill West",2,0
7,1,0,0,0,43.713756,-79.490074,1,0,0,0,"Downsview,North Park,Upwood Park",2,0
8,1,0,0,0,43.750072,-79.295849,0,0,0,0,"Maryvale,Wexford",2,0
9,2,4,13,2,43.640816,-79.381752,2,2,2,1,"Harbourfront East,Toronto Islands,Union Station",1,0


In [60]:
feature_df[feature_df['Yoga Studio']>0]

,Bakery,Café,Coffee Shop,Italian Restaurant,Lat,Long,Park,Pizza Place,Restaurant,Sandwich Place,neighborhood,Cluster,Yoga Studio
1,1,2,3,1,43.647927,-79.419750,1,2,2,0,"Little Portugal,Trinity",0,1
4,1,4,12,4,43.657952,-79.387383,1,0,0,3,Central Bay Street,1,1
20,0,1,2,0,43.715383,-79.405678,1,0,1,0,North Toronto West,2,1
34,1,1,4,3,43.679557,-79.352188,0,1,1,0,"The Danforth West,Riverdale",0,1
35,3,3,9,0,43.654260,-79.360636,3,0,1,0,"Harbourfront,Regent Park",3,1
75,0,1,9,1,43.662301,-79.389494,2,0,1,1,Queen's Park,3,1
78,0,2,6,1,43.665860,-79.383160,2,1,3,0,Church and Wellesley,3,1
83,2,4,3,2,43.659526,-79.340923,1,0,0,1,Studio District,0,1
84,0,0,1,0,43.705369,-79.349372,1,1,0,1,Thorncliffe Park,2,1
98,1,2,2,1,43.636847,-79.428191,0,0,1,0,"Brockton,Exhibition Place,Parkdale Village",0,1


In [61]:
# A lot of these are in cluster 3 and 2
# So what this means is that other neighborhoods in cluster 2 and 3 might benefit most from a yoga studio!!!

In [62]:
cluster_2_df = feature_df[feature_df['Cluster']==2].reset_index(drop=True)
cluster_3_df = feature_df[feature_df['Cluster']==3].reset_index(drop=True)

In [63]:
cluster_2_df

,Bakery,Café,Coffee Shop,Italian Restaurant,Lat,Long,Park,Pizza Place,Restaurant,Sandwich Place,neighborhood,Cluster,Yoga Studio
0,0,0,0,0,43.706748,-79.594054,0,0,0,0,Northwest,2,0
1,0,0,0,0,43.685347,-79.338106,2,0,0,0,East Toronto,2,0
2,0,0,0,0,43.696948,-79.411307,0,0,0,0,"Forest Hill North,Forest Hill West",2,0
3,1,0,0,0,43.713756,-79.490074,1,0,0,0,"Downsview,North Park,Upwood Park",2,0
4,1,0,0,0,43.750072,-79.295849,0,0,0,0,"Maryvale,Wexford",2,0
5,0,0,0,0,43.689574,-79.383160,1,0,0,0,"Moore Park,Summerhill East",2,0
6,1,2,0,1,43.661608,-79.464763,1,0,0,0,"High Park,The Junction South",2,0
7,0,0,1,0,43.767980,-79.487262,0,0,0,0,"Northwood Park,York University",2,0
8,0,0,1,0,43.718518,-79.464763,0,0,0,0,"Lawrence Heights,Lawrence Manor",2,0
9,0,0,0,0,43.763573,-79.188711,0,1,0,0,"Guildwood,Morningside,West Hill",2,0


In [64]:
cluster_3_df

,Bakery,Café,Coffee Shop,Italian Restaurant,Lat,Long,Park,Pizza Place,Restaurant,Sandwich Place,neighborhood,Cluster,Yoga Studio
0,2,0,5,0,43.778517,-79.346556,0,0,1,1,"Fairview,Henry Farm,Oriole",3,0
1,3,3,9,0,43.654260,-79.360636,3,0,1,0,"Harbourfront,Regent Park",3,1
2,2,3,8,2,43.657162,-79.378937,0,2,2,1,"Ryerson,Garden District",3,0
3,0,1,9,1,43.662301,-79.389494,2,0,1,1,Queen's Park,3,1
4,0,2,6,1,43.665860,-79.383160,2,1,3,0,Church and Wellesley,3,1


In [65]:
test = cluster_2_df[cluster_2_df['Yoga Studio']==0]
test = test.append(cluster_3_df[cluster_3_df['Yoga Studio']==0])
towns_to_open_up_yoga_studios = list(set(test['neighborhood']))
towns_to_open_up_yoga_studios

['Northwest',
 'Caledonia-Fairbanks',
 'Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West',
 "Humber Bay,King's Mill Park,Kingsway Park South East,Mimico NE,Old Mill South,The Queensway East,Royal York South East,Sunnylea",
 'East Toronto',
 'Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown',
 'Bathurst Manor,Downsview North,Wilson Heights',
 'Fairview,Henry Farm,Oriole',
 'Forest Hill North,Forest Hill West',
 'Dovercourt Village,Dufferin',
 'Downsview,North Park,Upwood Park',
 'Parkdale,Roncesvalles',
 'Scarborough Village',
 'Woodbine Gardens,Parkview Hill',
 'Woodbine Heights',
 'Cedarbrae',
 'Alderwood,Long Branch',
 'Emery,Humberlea',
 'Glencairn',
 'Maryvale,Wexford',
 'Cliffcrest,Cliffside,Scarborough Village West',
 'Canada Post Gateway Processing Centre',
 'Bayview Village',
 'Weston',
 'Parkwoods',
 'Rosedale',
 "Clarks Corners,Sullivan,Tam O'Shanter",
 'The Junction North,Runnymede',
 'Moore Park,Summerhill E

In [70]:
yoga_neighborhoods_df = pd.DataFrame(towns_to_open_up_yoga_studios, columns = ['Neighborhood'])

In [81]:
yoga_neighborhoods_df.tail(41)

,Neighborhood
29,"CFB Toronto,Downsview East"
30,"Northwood Park,York University"
31,"Flemingdon Park,Don Mills South"
32,"Birch Cliff,Cliffside West"
33,"Lawrence Heights,Lawrence Manor"
34,Willowdale West
35,"Del Ray,Keelesdale,Mount Dennis,Silverthorn"
36,"Guildwood,Morningside,West Hill"
37,"Dorset Park,Scarborough Town Centre,Wexford He..."
38,"The Beaches West,India Bazaar"
